1. Выбираем часть датасета где acnum = VQ-BDQ, pos = 1, phase = TAKEOFF

In [1]:
import pandas as pd

egtm_raw = pd.read_csv('./EGTM.csv', parse_dates=['RECORDED_DT'])

In [2]:
n1_raw = pd.read_csv('./Data N1.csv', parse_dates=['RECORDED_DT'])

n1_cols = ['acnum', 'pos', 'recordedts', 'n1_delt']

n1_clean = n1_raw.rename(columns={
  'AIRCRAFT': 'acnum',
  'POSITION': 'pos',
  'RECORDED_DT': 'recordedts',
  'DELTA N1 (Actual vs Cmd)': 'n1_delt'
})[n1_cols]

n1_clean['recordedts'] = pd.to_datetime(n1_clean['recordedts'], format='mixed')


n1 = n1_clean.copy()

n1.notna().sum()

acnum         4631
pos           4631
recordedts    4631
n1_delt       4270
dtype: int64

In [3]:
n2_raw = pd.read_csv('./Data N2.csv', parse_dates=['RECORDED_DT'])

n2_cols = ['acnum', 'pos', 'recordedts', 'n2_delt']

n2_clean = n2_raw.rename(columns={
  'AIRCRAFT': 'acnum',
  'POSITION': 'pos',
  'RECORDED_DT': 'recordedts',
  'N2 Delt': 'n2_delt'
})[n2_cols]

n2_clean['recordedts'] = pd.to_datetime(n2_clean['recordedts'], format='mixed')


n2 = n2_clean.copy()

n2.notna().sum()

acnum         4394
pos           4394
recordedts    4394
n2_delt       4269
dtype: int64

In [4]:
egtm_cols = ['acnum', 'pos', 'recordedts', 'egtm']

egtm = egtm_raw.rename(columns={
  'AIRCRAFT': 'acnum',
  'POSITION': 'pos',
  'RECORDED_DT': 'recordedts',
  'RWCTOMAR': 'egtm'
})[egtm_cols].copy()

egtm['recordedts'] = pd.to_datetime(egtm['recordedts'], format='mixed')
egtm['reportts'] = egtm['recordedts'] 

egtm.tail(3)

,acnum,pos,recordedts,egtm,reportts
4341,VQ-BGU,2,2020-10-02 02:04:00,17.079,2020-10-02 02:04:00
4342,VQ-BGU,2,2020-10-02 17:24:00,16.755,2020-10-02 17:24:00
4343,VQ-BGU,2,2020-11-02 02:09:00,16.916,2020-11-02 02:09:00


In [5]:
egtm = pd.merge(egtm, n1, on=['acnum', 'pos', 'recordedts'], how='left')
egtm = pd.merge(egtm, n2, on=['acnum', 'pos', 'recordedts'], how='left')

In [6]:
egtm

,acnum,pos,recordedts,egtm,reportts,n1_delt,n2_delt
0,VQ-BDQ,1,2018-05-20 19:36:47,132.144,2018-05-20 19:36:47,NaN,NaN
1,VQ-BDQ,1,2018-05-22 01:48:50,127.592,2018-05-22 01:48:50,NaN,NaN
2,VQ-BDQ,1,2018-05-23 19:51:23,128.712,2018-05-23 19:51:23,NaN,NaN
3,VQ-BDQ,1,2018-05-25 01:07:05,129.118,2018-05-25 01:07:05,NaN,NaN
4,VQ-BDQ,1,2018-05-26 10:41:08,129.008,2018-05-26 10:41:08,NaN,NaN
...,...,...,...,...,...,...,...
4339,VQ-BGU,2,2020-06-02 13:59:00,17.551,2020-06-02 13:59:00,-0.080002,-0.408904
4340,VQ-BGU,2,2020-09-02 02:18:00,17.467,2020-09-02 02:18:00,-0.059998,-0.545474
4341,VQ-BGU,2,2020-10-02 02:04:00,17.079,2020-10-02 02:04:00,-0.040001,-0.313053
4342,VQ-BGU,2,2020-10-02 17:24:00,16.755,2020-10-02 17:24:00,-0.059998,-0.524035


In [7]:
neo_raw = pd.read_csv(
  './a320_a321_neo_full_acms_parameters.csv', 
  parse_dates=['reportts', 'processedts'], 
  low_memory=False
)
neo = neo_raw \
[
  # (neo_raw['acnum'] == 'VQ-BDQ') & 
  # (neo_raw['pos'] == 1) & 
  (neo_raw['reportname'] == 'A320 ENGINE TAKE OFF REPORT') |
  (neo_raw['reportname'] == 'A321 ENGINE TAKE OFF REPORT')
].copy()

neo.tail(3)

,acnum,reportname,reportts,file_path,pos,fltdes,dep,arr,ivs12,ibe,...,vsva,w14,processedts,pf,wai,nai,prv,hpv,xf,reason
94649,VQ-BGU,A321 ENGINE TAKE OFF REPORT,2022-03-13 15:08:36,adw-data/aircraft/pdm/data/qar/binary/A321NEO/...,2,1055.0,UUDD,UWKD,NaN,NaN,...,0.0,1177.0,2023-09-22 22:56:31.497,0.51,0.0,0.0,1.0,0.0,0.0,NaN
94650,VQ-BGU,A321 ENGINE TAKE OFF REPORT,2022-03-22 21:43:06,adw-data/aircraft/pdm/data/qar/binary/A321NEO/...,1,1227.0,UUDD,ULMM,NaN,NaN,...,0.0,1293.0,2023-09-22 22:56:37.929,0.01,0.0,0.0,0.0,0.0,0.0,NaN
94651,VQ-BGU,A321 ENGINE TAKE OFF REPORT,2022-03-22 21:43:06,adw-data/aircraft/pdm/data/qar/binary/A321NEO/...,2,1227.0,UUDD,ULMM,NaN,NaN,...,0.0,1292.0,2023-09-22 22:56:37.929,0.03,0.0,0.0,0.0,0.0,0.0,NaN


In [8]:
from typing import List

def merge_by_date(left: pd.DataFrame, right: pd.DataFrame):
  left.sort_values('reportts', inplace=True)
  right.sort_values('reportts', inplace=True)

  left = left.set_index('reportts')
  right = right.set_index('reportts')

  right_indexed = \
      right.reindex(left.index, method='ffill').reset_index()
  return pd.merge(left, right_indexed, on=['acnum', 'pos','reportts'], suffixes=(None, '_'))

def merge_fleet(acms: pd.DataFrame, target: pd.DataFrame, fleet: List[str], positions: List[int] = [1, 2]):
  result = []
  
  for acnum in fleet:
    for pos in positions:
      left = acms[(acms['acnum'] == acnum) & (acms['pos'] == pos)].copy()
      right = target[(target['acnum'] == acnum) & (target['pos'] == pos)].copy()
      merged = merge_by_date(left, right)
      result.append(merged)

  return pd.concat(result, ignore_index=True)



In [9]:

merged = merge_fleet(neo, egtm, fleet=['VQ-BDQ', 'VQ-BGU'])

merged[['reportts','acnum','pos','recordedts','egtm', 'n1_delt', 'n2_delt']]


,reportts,acnum,pos,recordedts,egtm,n1_delt,n2_delt
0,2018-05-20 02:58:38,VQ-BDQ,1,2018-05-12 23:14:00,118.644,-0.079994,-0.111004
1,2018-05-20 19:36:46,VQ-BDQ,1,2018-05-12 23:14:00,118.644,-0.079994,-0.111004
2,2018-05-22 01:48:49,VQ-BDQ,1,2018-05-20 19:36:47,132.144,NaN,NaN
3,2018-05-23 19:51:22,VQ-BDQ,1,2018-05-22 01:48:50,127.592,NaN,NaN
4,2018-05-25 01:07:04,VQ-BDQ,1,2018-05-23 19:51:23,128.712,NaN,NaN
...,...,...,...,...,...,...,...
11469,2022-04-22 22:16:10,VQ-BGU,2,2020-11-02 02:09:00,16.916,-0.089996,-0.279018
11470,2022-04-23 04:52:35,VQ-BGU,2,2020-11-02 02:09:00,16.916,-0.089996,-0.279018
11471,2022-04-23 10:24:44,VQ-BGU,2,2020-11-02 02:09:00,16.916,-0.089996,-0.279018
11472,2022-04-23 16:02:53,VQ-BGU,2,2020-11-02 02:09:00,16.916,-0.089996,-0.279018


In [10]:
diff = (merged['reportts'] - merged['recordedts']) \
  .astype('timedelta64[s]').astype('int')

In [11]:
merged_closest = \
  merged[diff <= (1 * 60 * 60)] \
    .drop(columns=['file_path', 'reportname', 'processedts'])


In [12]:
merged_closest

,acnum,reportts,pos,fltdes,dep,arr,ivs12,ibe,iaie,iai,...,wai,nai,prv,hpv,xf,reason,recordedts,egtm,n1_delt,n2_delt
16,VQ-BDQ,2018-06-10 20:40:23,1,3338.0,ULLI,UNNT,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,NaN,2018-06-10 19:51:00,119.794,-0.089996,-0.231438
18,VQ-BDQ,2018-06-13 09:03:42,1,3312.0,URSS,UNNT,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,NaN,2018-06-13 09:03:42,124.451,NaN,NaN
29,VQ-BDQ,2018-06-25 14:14:43,1,3337.0,UNNT,ULLI,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,NaN,2018-06-25 14:14:43,122.700,NaN,NaN
133,VQ-BDQ,2019-10-06 10:54:23,1,6342.0,RJAA,UIII,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,2019-10-06 10:22:00,114.588,-0.059998,-0.218007
142,VQ-BDQ,2019-10-09 03:35:56,1,6321.0,UIII,VHHH,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,2019-10-09 02:52:00,112.184,-0.070000,-0.226165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9224,VQ-BGU,2019-12-12 04:33:05,2,5784.0,VVCR,UNNT,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,NaN,2019-12-12 04:33:00,21.725,0.099998,-0.256320
9270,VQ-BGU,2020-02-02 06:22:04,2,5555.0,UNNT,UTDL,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,2020-02-02 06:22:00,17.509,-0.100006,-0.226369
9272,VQ-BGU,2020-02-02 17:40:16,2,5783.0,UNNT,VVCR,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,NaN,2020-02-02 17:40:00,17.318,-0.090004,-0.360318
9530,VQ-BGU,2020-08-01 14:30:20,2,2074.0,URKK,UUDD,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,NaN,2020-08-01 13:51:00,19.622,0.030006,-0.608787


In [13]:
merged_closest.to_csv('./sample-ALL-NEO-egtm.csv', index=False)